### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js



In [1]:
import requests

In [2]:
class Rate:
    def __init__(self, format_='value'):
        self.format = format_
        self.r = None
        self.show_difference = False
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        if self.r is None: 
            self.r = self.get_data()
        
        return self.r.json()['Valute']
    
    def get_data(self):
        return requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.show_difference and currency in ["EUR","USD"]:
                return response[currency]['Value'] - response[currency]['Previous']
            elif self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def diff(self, show_difference):
        self.show_difference = show_difference
        return self
        
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')
    
    def get_max_rate_currency(self):
        currencyRates = {currency[1]["Value"]:currency[1]["Name"] for currency in self.exchange_rates().items()}
        return currencyRates[max(currencyRates.keys())]

#### см. ниже

### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.



In [3]:
rateObj = Rate()

In [4]:
print("Курс Евро:", rateObj.eur())
print("Изменение курса Евро:",rateObj.diff(True).eur())
print("Максимальный курс - у",rateObj.get_max_rate_currency())

rateObj.format = "full"
print("Полные данные по Евро:",rateObj.make_format("EUR"))

Курс Евро: 71.1918
Изменение курса Евро: 0.18370000000000175
Максимальный курс - у Датских крон
Полные данные по Евро: {'ID': 'R01239', 'NumCode': '978', 'CharCode': 'EUR', 'Nominal': 1, 'Name': 'Евро', 'Value': 71.1918, 'Previous': 71.0081}


In [5]:
rateObj.format = "value"
print("Курс бразильского реала:", rateObj.brl())
print("Изменение в diff не влияет на Бразильсктй Реал: ",rateObj.diff(True).brl())

rateObj.format = "full"
print("Полные данные по бразильскому Реалу:",rateObj.make_format("BRL"))

Курс бразильского реала: 15.9925
Изменение в diff не влияет на Бразильсктй Реал:  15.9925
Полные данные по бразильскому Реалу: {'ID': 'R01115', 'NumCode': '986', 'CharCode': 'BRL', 'Nominal': 1, 'Name': 'Бразильский реал', 'Value': 15.9925, 'Previous': 15.9759}


### Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

In [6]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [7]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

#### Класс Designer

In [8]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        if (self.seniority + (self.awards*2)) % 7 == 0:
            self.grade_up()
        
        return self.publish_grade()
    
    def recieved_award(self):
        self.awards += 1
        print(self.name, ' recieved international award!')

In [9]:
elena = Designer('Елена', seniority=0, awards=2)
roberto = Designer("Роберто", seniority=0, awards=0)

In [10]:
for step in range(20):
    print("\n#", step)
    if step > 0:
        if step % 4 == 0: elena.recieved_award()
        if step % 2 == 0: roberto.recieved_award() # Робертно явно очень талантлив...
    elena.check_if_it_is_time_for_upgrade()
    roberto.check_if_it_is_time_for_upgrade()


# 0
Елена 1
Роберто 1

# 1
Елена 1
Роберто 1

# 2
Роберто  recieved international award!
Елена 2
Роберто 1

# 3
Елена 2
Роберто 1

# 4
Елена  recieved international award!
Роберто  recieved international award!
Елена 2
Роберто 1

# 5
Елена 2
Роберто 1

# 6
Роберто  recieved international award!
Елена 2
Роберто 1

# 7
Елена 3
Роберто 2

# 8
Елена  recieved international award!
Роберто  recieved international award!
Елена 3
Роберто 2

# 9
Елена 3
Роберто 2

# 10
Роберто  recieved international award!
Елена 3
Роберто 3

# 11
Елена 3
Роберто 3

# 12
Елена  recieved international award!
Роберто  recieved international award!
Елена 3
Роберто 3

# 13
Елена 3
Роберто 3

# 14
Роберто  recieved international award!
Елена 3
Роберто 3

# 15
Елена 3
Роберто 3

# 16
Елена  recieved international award!
Роберто  recieved international award!
Елена 3
Роберто 3

# 17
Елена 3
Роберто 3

# 18
Роберто  recieved international award!
Елена 3
Роберто 3

# 19
Елена 3
Роберто 3
